# Memory profiling - p122

In [1]:
import numpy as np
from imp import reload

%load_ext line_profiler

In [2]:
%%writefile _simulation.py

import numpy as np

def step(*shape):
    # Create random vector with +1 or -1
    return (2 * (np.random.random_sample(shape) < 0.5) - 1)

def simulate(iterations, n=10000):
    s = step(iterations, n)
    x = np.cumsum(s, axis=0)
    bins = np.arange(-30, 30, 1)
    y = np.vstack([np.histogram(x[i,:], bins)[0]
                  for i in range(iterations)])
    return y

Overwriting _simulation.py


In [3]:
import _simulation
reload(_simulation)

<module '_simulation' from '/home2/yannick2/github/Spikes/Python/_simulation.py'>

In [4]:
%lprun -T lprof0 -f _simulation.simulate _simulation.simulate(500)


*** Profile printout saved to text file 'lprof0'. 


In [5]:
! cat lprof0

Timer unit: 1e-06 s

Total time: 0.27732 s
File: /home2/yannick2/github/Spikes/Python/_simulation.py
Function: simulate at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def simulate(iterations, n=10000):
     9         1        74061  74061.0     26.7      s = step(iterations, n)
    10         1        23803  23803.0      8.6      x = np.cumsum(s, axis=0)
    11         1           22     22.0      0.0      bins = np.arange(-30, 30, 1)
    12         1            3      3.0      0.0      y = np.vstack([np.histogram(x[i,:], bins)[0]
    13         1       179430 179430.0     64.7                    for i in range(iterations)])
    14         1            1      1.0      0.0      return y

In [6]:
%load_ext memory_profiler

import gc

gc.collect()

%mprun -T mprof0 -f _simulation.simulate _simulation.simulate(50, int(1e5))
! cat mprof0



*** Profile printout saved to text file mprof0. 
Filename: /home2/yannick2/github/Spikes/Python/_simulation.py

Line #    Mem usage    Increment   Line Contents
     8     46.8 MiB      0.0 MiB   def simulate(iterations, n=10000):
     9     89.5 MiB     42.8 MiB       s = step(iterations, n)
    10    127.5 MiB     38.0 MiB       x = np.cumsum(s, axis=0)
    11    127.5 MiB      0.0 MiB       bins = np.arange(-30, 30, 1)
    12    127.5 MiB      0.0 MiB       y = np.vstack([np.histogram(x[i,:], bins)[0]
    13    127.5 MiB      0.0 MiB                     for i in range(iterations)])
    14    127.5 MiB      0.0 MiB       return y

In [7]:
%memit y = np.random.randn(1000,10000)

peak memory: 127.73 MiB, increment: 76.33 MiB


In [8]:
x = np.random.randn(10000, 1000)
%memit y = None
%time %memit x = x * 2
%time %memit x *= 2
%memit x[10,:] = x[11,:] * 2
%memit y = x[11,:] * 2
%memit y = y * 3
%memit x[:] = x[:] * 2

peak memory: 127.75 MiB, increment: -76.29 MiB
peak memory: 199.21 MiB, increment: 71.46 MiB
CPU times: user 52.5 ms, sys: 20.8 ms, total: 73.3 ms
Wall time: 184 ms
peak memory: 127.76 MiB, increment: 0.00 MiB
CPU times: user 35.2 ms, sys: 41.6 ms, total: 76.8 ms
Wall time: 191 ms
peak memory: 127.76 MiB, increment: 0.00 MiB
peak memory: 127.76 MiB, increment: 0.00 MiB
peak memory: 127.91 MiB, increment: 0.15 MiB
peak memory: 204.21 MiB, increment: 76.30 MiB


In [20]:
import scipy 
import scipy.sparse

N = 4000
s = 4000

def bigsparse (N, s):
    x = np.random.randint(0, N, s)
    y = np.random.randint(0, N, s)
    return scipy.sparse.csr_matrix((np.ones(s), (x,y)), (N,N))

sparse1 = None
sparse2 = None
%memit sparse1 = bigsparse(N, s)
foo = None
%time %memit foo = sparse1 * sparse1
%time print((foo != 0).sum())

%memit sparse2 = bigsparse(N, s * 100)
foo = None
%time %memit foo = sparse2 * sparse2
%time print((foo != 0).sum())

%memit nonsparse_m = sparse2.todense()
foo = None
%time %memit foo = nonsparse_m * nonsparse_m
%time print((foo != 0).sum())

peak memory: 360.95 MiB, increment: 0.00 MiB
peak memory: 238.88 MiB, increment: 0.00 MiB
CPU times: user 25.3 ms, sys: 19.2 ms, total: 44.5 ms
Wall time: 170 ms
3935
CPU times: user 547 µs, sys: 62 µs, total: 609 µs
Wall time: 498 µs
peak memory: 238.88 MiB, increment: 0.00 MiB
peak memory: 405.99 MiB, increment: 167.12 MiB
CPU times: user 349 ms, sys: 14.9 ms, total: 364 ms
Wall time: 468 ms
14602353
CPU times: user 775 ms, sys: 12.4 ms, total: 787 ms
Wall time: 787 ms
peak memory: 525.86 MiB, increment: 119.87 MiB
peak memory: 360.95 MiB, increment: 122.07 MiB
CPU times: user 7.77 s, sys: 107 ms, total: 7.88 s
Wall time: 2.57 s
14602353
CPU times: user 62.3 ms, sys: 3.81 ms, total: 66.1 ms
Wall time: 33.1 ms
